<a href="https://colab.research.google.com/github/eringgao/lucy_listings/blob/main/lucyslistings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Declarations

In [3]:
import pandas as pd
import numpy as np
from geopy import distance
import requests
import json
import datetime
import math
import itertools
import json
from abc import ABC, abstractmethod

# Data Processsing

In [4]:
df = pd.read_csv("Data/rapid_data.csv")
cmu_df = pd.read_csv("Data/CMU_listing_data.csv")

df = df.drop(columns = ["currency", "daysOnZillow", "homeStatusForHDP", "zpid", "datePriceChanged", "priceChange", "priceReduction", "taxAssessedValue", "isFeatured", "isNonOwnerOccupied", "priceForHDP", "shouldHighlight", "homeStatus", "isPreforeclosureAuction", "isPremierBuilder", "isZillowOwned", "isRentalWithBasePrice", "isUnmappable"])

print(df)

    Unnamed: 0  bathrooms  bedrooms           city country       homeType  \
0            0        2.0         4     Pittsburgh     USA      APARTMENT   
1            1        1.0         2  Mt Washington     USA      APARTMENT   
2            2        1.5         3     Pittsburgh     USA  SINGLE_FAMILY   
3            3        2.0         4     Pittsburgh     USA  SINGLE_FAMILY   
4            4        1.0         1     Pittsburgh     USA      APARTMENT   
5            5        1.0         3     Pittsburgh     USA  SINGLE_FAMILY   
6            6        1.0         2     Pittsburgh     USA  SINGLE_FAMILY   
7            7        3.0         5     Pittsburgh     USA  SINGLE_FAMILY   
8            8        1.5         2     Pittsburgh     USA      TOWNHOUSE   
9            9        2.0         3     Pittsburgh     USA  SINGLE_FAMILY   
10          10        2.0         3     Pittsburgh     USA      APARTMENT   
11          11        1.0         2     Pittsburgh     USA      APARTMENT   

In [5]:
'''
  Create an OOP structure for how we want to store all of our data.
  Example format: 
'''
class Listing: # abstraction?
  def get_type(self) -> str: 
    # Manual, Zillow, CMU
    pass 
  def get_housing_info(self) -> str: 
    # String data (paragraphs?) about housing
    pass
  def get_address(self) -> dict: 
    pass
  def distance_from_cmu(self) -> float: 
    pass

class SchoolListing(Listing): 
  def __init__(self, address): 
    # Some stuff here
    self.get_type = "CMU"
    # self.distance_from_cmu 


In [6]:
addresses = df.loc[:, "streetAddress"]
city = df.loc[:, "city"]
state = df.loc[:, "state"]
house_type = df.loc[:, "homeType"]
lat = df.loc[:, "latitude"]
longi = df.loc[:, "longitude"]
price = df.loc[:, "price"]
zip = df.loc[:, "zipcode"]
cmu_address = (40.4433, -79.944023)
distance_df = pd.DataFrame(columns = ['addr', 'Latitude', 'Longitude'])

distance_df['addr'] = addresses
distance_df['city'] = city
distance_df['state'] = state
distance_df['house_type'] = house_type
distance_df['Latitude'] = lat
distance_df['Longitude'] = longi
distance_df['price'] = price
distance_df['zip'] = zip

In [7]:
print(distance_df)

                             addr   Latitude  Longitude           city state  \
0               2353 Sidney St #2  40.429380 -79.971140     Pittsburgh    PA   
1              441 Edgemont St #2  40.422600 -79.998880  Mt Washington    PA   
2                   200 S 15th St  40.424866 -79.983440     Pittsburgh    PA   
3                 5317 Forbes Ave  40.440155 -79.934940     Pittsburgh    PA   
4              4711 Maripoe St #1  40.455498 -79.949670     Pittsburgh    PA   
5                   1625 Hybla St  40.474262 -80.037970     Pittsburgh    PA   
6             3202 Hazelhurst Ave  40.383247 -79.965300     Pittsburgh    PA   
7                4245 Parkman Ave  40.448345 -79.957280     Pittsburgh    PA   
8              1007 Galveston Ave  40.453010 -80.017100     Pittsburgh    PA   
9                 11370 Althea Rd  40.468670 -79.832700     Pittsburgh    PA   
10              333 Lehigh Ave #2  40.456150 -79.926100     Pittsburgh    PA   
11  1465 Old Steubenville Pike #B  40.45

In [8]:
walking = []
driving = []
for i in range(len(addresses)):
  d = distance.distance((distance_df.loc[i, "Latitude"], distance_df.loc[i, "Longitude"]), (cmu_address[0],cmu_address[1]))
  walking.append(d.miles)
  r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{distance_df.loc[i, "Longitude"]},{distance_df.loc[i, "Latitude"]};{cmu_address[1]},{cmu_address[0]}?overview=false""")
  route_1 = json.loads(r.content)["routes"][0]
  driving.append(route_1["distance"]*0.000621371)
distance_df["walk"] = walking
distance_df["drive"] = driving

In [9]:
d_df = distance_df.copy()

In [10]:
num_entries = len(distance_df['addr'])

list_type = ['Zillow' for _ in range(num_entries)]
distance_df['list_type'] = list_type

print(distance_df)

                             addr   Latitude  Longitude           city state  \
0               2353 Sidney St #2  40.429380 -79.971140     Pittsburgh    PA   
1              441 Edgemont St #2  40.422600 -79.998880  Mt Washington    PA   
2                   200 S 15th St  40.424866 -79.983440     Pittsburgh    PA   
3                 5317 Forbes Ave  40.440155 -79.934940     Pittsburgh    PA   
4              4711 Maripoe St #1  40.455498 -79.949670     Pittsburgh    PA   
5                   1625 Hybla St  40.474262 -80.037970     Pittsburgh    PA   
6             3202 Hazelhurst Ave  40.383247 -79.965300     Pittsburgh    PA   
7                4245 Parkman Ave  40.448345 -79.957280     Pittsburgh    PA   
8              1007 Galveston Ave  40.453010 -80.017100     Pittsburgh    PA   
9                 11370 Althea Rd  40.468670 -79.832700     Pittsburgh    PA   
10              333 Lehigh Ave #2  40.456150 -79.926100     Pittsburgh    PA   
11  1465 Old Steubenville Pike #B  40.45

# OOP Structuring

In [11]:
class Listing(ABC): # abstraction?
  def __init__(self, addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None): 
    self.address = addr_dict
    self.house_type = h_type 
    self.walk = walk_time
    self.drive = drive_dist
    self.price = price
    self.description = description

  def get_housing_info(self): 
    return self.housing_info

  def update_housing_info(self, new_info): 
    # pass
    self.housing_info = new_info

  def get_address(self): 
    return self.address

  def get_drive_distance(self): 
    return self.drive

  @abstractmethod
  def get_list_type(self): 
    pass

  # def distance_from_cmu(self) -> float: 
  #   pass

# class SchoolListing(Listing): 


class ZillowListing(Listing): 
  def __init__(self, addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None, zlink=None): 
    super().__init__(addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None)
    self.zillow_link = zlink

  def get_list_type(self): 
    return "Zillow"
  
  def get_zillow_link(self):
    return self.zillow_link

  def update_zillow_link(self, new_link):
    self.zillow_link = new_link


  


In [12]:
zill_listings = [ZillowListing({'addr':row.addr, 'city':row.city, 'state':row.state, 'zipcode':row.zip}, row.price, row.walk, row.drive, row.house_type) for _, row in distance_df.iterrows()]

In [13]:
for listing in zill_listings: 
  print(listing.get_address())

{'addr': '2353 Sidney St #2', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15203}
{'addr': '441 Edgemont St #2', 'city': 'Mt Washington', 'state': 'PA', 'zipcode': 15211}
{'addr': '200 S 15th St', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15203}
{'addr': '5317 Forbes Ave', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15217}
{'addr': '4711 Maripoe St #1', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15213}
{'addr': '1625 Hybla St', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15212}
{'addr': '3202 Hazelhurst Ave', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15227}
{'addr': '4245 Parkman Ave', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15213}
{'addr': '1007 Galveston Ave', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15233}
{'addr': '11370 Althea Rd', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15235}
{'addr': '333 Lehigh Ave #2', 'city': 'Pittsburgh', 'state': 'PA', 'zipcode': 15232}
{'addr': '1465 Old Steubenville Pike #B', 'city': 'Pittsburgh', 'state

In [14]:

class CMUListing(Listing):
    def __init__(self, addr_dict, grade, building_name, room_type, bathroom_type, price, ac):
        super().__init__(addr_dict, price)
        self.building_name = building_name
        self.room_type = room_type
        self.bathroom_type = bathroom_type
        self.ac = ac   
    def get_list_type(self):
        return "CMU On-Campus"
class ManualListing(Listing):
    def __init__(self, first_name, last_name, contact, availability, addr_dict, bathrooms, bedrooms, home_type):
        super().__init__(addr_dict, price)
        self.first_name = first_name
        self.last_name = last_name
        self.contact = contact
        self.availability = availability 
        self.bathrooms = bathrooms
        self.bedrooms = bedrooms
        self.home_type = home_type
    def get_list_type(self):
        return "User Posting"

In [25]:
df_cmu = pd.read_csv("Data/CMU_listing_data.csv")
address = []
cities = []
states = []
zipcodes = []
for i in df_cmu.loc[:, "Address"]:
    if(i == "Corner of Morewood Avenue and Forbes Avenue"):
        i = "1065 Morewood Ave, Pittsburgh, PA 15213"
    if(i == "Margaret Morrison Corridor"):
        i = "5143 Forbes Ave, Pittsburgh, PA 15213"
    splits = i.split(',')
    address.append(splits[0])
    cities.append(splits[1])
    states.append(splits[2].split(" ")[0])
    zipcodes.append(splits[2].split(" ")[1])
cmu_listings = []
i = 0
for _, row in df_cmu.iterrows():
    cmu_listings.append(CMUListing({'addr':address[i], 'city':cities[i], 'state':states[i], 'zipcode':zipcodes[i]}, row.For, row.Building, row.Room, row.Bathroom, row.Cost, row.AC))
    i+=1

In [30]:
from geopy.geocoders import Nominatim

longs = []
lats = []
geolocator = Nominatim(user_agent="my_request")

for i in df_cmu.loc[:, "Address"]:
    if(i == "Corner of Morewood Avenue and Forbes Avenue"):
        i = "1065 Morewood Ave, Pittsburgh, PA 15213"
    if(i == "Margaret Morrison Corridor"):
        i = "5143 Forbes Ave, Pittsburgh, PA 15213"
    #making an instance of Nominatim class
    #applying geocode method to get the location
    location = geolocator.geocode(i)
    if location == None: 
        longitude = -79.944023
        latitude = 40.443336
    else:
        longitude = location.longitude
        latitude = location.latitude
    #printing address and coordinates
    longs.append(longitude)
    lats.append(latitude)
df_cmu["Longitude"] = longs
df_cmu["Latitude"] = lats
df_cmu.to_csv("Data/CMU_listing_data.csv")


In [ ]:
# to read csv file named "samplee"

In [27]:
all_listings = cmu_listings + zill_listings

j_str = json.dumps([ob.__dict__ for ob in all_listings])

print(j_str)zz

[{"address": {"addr": "5126 Margaret Morrison St", "city": " Pittsburgh", "state": "", "zipcode": "PA"}, "house_type": null, "walk": null, "drive": null, "price": "5,741", "description": null, "building_name": "Boss House", "room_type": "Semi-Suite Double", "bathroom_type": "Shared", "ac": "Window"}, {"address": {"addr": "5126 Margaret Morrison St", "city": " Pittsburgh", "state": "", "zipcode": "PA"}, "house_type": null, "walk": null, "drive": null, "price": "6,665", "description": null, "building_name": "Boss House", "room_type": "Semi-Suite Single", "bathroom_type": "Shared", "ac": "Window"}, {"address": {"addr": "624 Clyde St", "city": " Pittsburgh", "state": "", "zipcode": "PA"}, "house_type": null, "walk": null, "drive": null, "price": "6,055", "description": null, "building_name": "Clyde House", "room_type": "Studio Apartment Double", "bathroom_type": "Shared", "ac": "No"}, {"address": {"addr": "624 Clyde St", "city": " Pittsburgh", "state": "", "zipcode": "PA"}, "house_type": n